# Capstone project

This Notebook will be used for the capstone project.

In [16]:
import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [17]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

data = pd.read_html(link)[0]
data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [18]:
# Cleaning the dataset

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df_ngbh_can = data.rename(columns={'Postal Code': 'PostalCode', 'Neighbourhood': 'Neighborhood'})

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_ngbh_can = df_ngbh_can[~(df_ngbh_can.Borough == 'Not assigned')].reset_index(drop=True)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df_ngbh_can['Neighborhood'] = [df_ngbh_can.Borough if x == 'Not assigned' else x for x in df_ngbh_can.Neighborhood]

df_ngbh_can.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [19]:

csv_root = 'http://cocl.us/Geospatial_data'
df_ltd_long = pd.read_csv(csv_root)


df_canada = df_ngbh_can.merge(df_ltd_long, left_on='PostalCode', right_on='Postal Code', how='inner')

# Dropping innecessary columns: Postal Code (duplicated)
df_canada.drop('Postal Code', axis=1, inplace=True)
df_canada.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Clustering

In [20]:
# Filtering by Boroughs that contain the word 'Toronto': Conforming the new df_toronto
df_toronto = df_canada[df_canada.Borough.str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [21]:
# Defining the function to obtain nearby venues: getNearbyVenues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# Defining the credentials needed to read the venues from Foursquare:
CLIENT_ID = 'ANNV5QVQNPQPHJWUIR1J2YAQ44WIIBFLJYP512NF1PG0WSRD' # your Foursquare ID
CLIENT_SECRET = '3JQHTEPR1ZQS02ZUYFOLLMH5ZXY1UIVYMJJKT14LL1OSYARG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ANNV5QVQNPQPHJWUIR1J2YAQ44WIIBFLJYP512NF1PG0WSRD
CLIENT_SECRET:3JQHTEPR1ZQS02ZUYFOLLMH5ZXY1UIVYMJJKT14LL1OSYARG


In [23]:
# Getting the venues with the pre-defined function using 'PostalCode'
toronto_venues = getNearbyVenues(df_toronto.Neighborhood, df_toronto.Latitude, df_toronto.Longitude, radius=1000)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(3182, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
3,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
4,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",48,48,48,48,48,48
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [26]:
# How many unique categories are there?
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,...,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
# Grouping each neighborhood by taking the mean of the frequency of occurrence of each category 
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,...,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,Berczy Park,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,0.00,0.00,0.020000,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.00,0.01,0.010000,0.00,0.02,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.00,0.00,...,0.00,0.000000,0.00,0.010000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.010000,0.010000,0.00,0.000000,0.01,0.000000,0.00,0.00,0.000000,0.00,0.00,0.010000,0.010000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.00,0.00,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.00,0.0000,0.0000,0.010000,0.00,0.00,0.00,0.00,0.020000,0.00,0.020000,0.000000,0.010000,0.00,0.000000,0.000000,0.040000,0.000000,0.050000,0.000000,0.00,0.00,0.000000,0.01,0.01,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.00,0.00,...,0.00,0.010000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.02,0.03,0.000000,0.000000,0.010000,0.00,0.000000,0.00,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.00,0.00,0.0000,0.0000,0.020833,0.00,0.00,0.00,0.00,0.000000,0.00,0.020833,0.000000,0.000000,0.00,0.020833,0.000000,0.041667,0.000000,0.020833,0.000000,0.00,0.00,0.020833,0.00,0.00,0.000000,0.00,0.00,0.020833,0.000000,0.000000,0.00,0.000000,0.020833,0.062500,0.00,0.00,...,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.020833,0.000000,0.00,0.000000,0.041667,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.020833,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.00,0.00,0.0625,0.0625,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,...,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.062500,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.062500,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000
4,Central Bay Street,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,0.00,0.00,0.020000,0.01,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.01,0.000000,0.00,0.00,0.000000,0.020000,0.000000,0.00,0.000000,0.010000,0.000000,0.00,0.02,...,0.01,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.030000,0.00,0.000000,0.00,0.000000,0.01,0.00,0.020000,0.00,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.020

In [29]:
# Defining a function to order the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
# Defining the n top venues and sorting them 
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Bakery,Restaurant,Gastropub,Cocktail Bar,Grocery Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Thrift / Vintage Store,Park
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Brewery,Coffee Shop,Fast Food Restaurant,Bakery,Sushi Restaurant,Italian Restaurant,French Restaurant,Steakhouse
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Harbor / Marina,Café,Track,Garden,Dog Run,Sculpture Garden,Sushi Restaurant,Park,Dance Studio
4,Central Bay Street,Coffee Shop,Café,Ramen Restaurant,Clothing Store,Diner,Sushi Restaurant,Park,Mexican Restaurant,Tea Room,Cosmetics Shop


In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 4, 0, 0, 1, 0, 2, 4, 0], dtype=int32)

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with df_toronto to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Restaurant,Café,Theater,Breakfast Spot,Sushi Restaurant,Bakery,Italian Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Park,Café,Sushi Restaurant,Japanese Restaurant,Pizza Place,Yoga Studio,Thai Restaurant,Bubble Tea Shop,Dance Studio
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Café,Gastropub,Japanese Restaurant,Diner,Theater,American Restaurant,Italian Restaurant,Ramen Restaurant,Hotel
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant,Italian Restaurant,Bakery,Theater,Clothing Store,Cocktail Bar
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Coffee Shop,Pizza Place,Beach,Park,Breakfast Spot,Japanese Restaurant,Burger Joint,Bakery,Bar


In [33]:
from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Park,Restaurant,Café,Theater,Breakfast Spot,Sushi Restaurant,Bakery,Italian Restaurant
1,Downtown Toronto,0,Coffee Shop,Park,Café,Sushi Restaurant,Japanese Restaurant,Pizza Place,Yoga Studio,Thai Restaurant,Bubble Tea Shop,Dance Studio
2,Downtown Toronto,0,Coffee Shop,Café,Gastropub,Japanese Restaurant,Diner,Theater,American Restaurant,Italian Restaurant,Ramen Restaurant,Hotel
6,Downtown Toronto,0,Coffee Shop,Café,Ramen Restaurant,Clothing Store,Diner,Sushi Restaurant,Park,Mexican Restaurant,Tea Room,Cosmetics Shop
20,Central Toronto,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Sushi Restaurant,Gym,Dessert Shop,Pizza Place,Fast Food Restaurant,Thai Restaurant
32,Downtown Toronto,0,Coffee Shop,Harbor / Marina,Café,Track,Garden,Dog Run,Sculpture Garden,Sushi Restaurant,Park,Dance Studio
37,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Park,Hotel,Sushi Restaurant,Yoga Studio,Dance Studio,Restaurant,Caribbean Restaurant,Ramen Restaurant


## Cluster 2

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,1,Pub,Coffee Shop,Pizza Place,Beach,Park,Breakfast Spot,Japanese Restaurant,Burger Joint,Bakery,Bar
7,Downtown Toronto,1,Korean Restaurant,Coffee Shop,Café,Grocery Store,Mexican Restaurant,Cocktail Bar,Vegetarian / Vegan Restaurant,Comedy Club,Ethiopian Restaurant,Ice Cream Shop
11,West Toronto,1,Café,Bar,Restaurant,Vegetarian / Vegan Restaurant,Bakery,Cocktail Bar,Pizza Place,Italian Restaurant,Asian Restaurant,Coffee Shop
12,East Toronto,1,Greek Restaurant,Coffee Shop,Café,Pub,Italian Restaurant,Fast Food Restaurant,Bank,Yoga Studio,Bakery,Furniture / Home Store
14,West Toronto,1,Café,Coffee Shop,Restaurant,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Thrift / Vintage Store,Park
15,East Toronto,1,Indian Restaurant,Grocery Store,Coffee Shop,Beach,Restaurant,Fast Food Restaurant,Bakery,Brewery,Gym,Park
17,East Toronto,1,Coffee Shop,Vietnamese Restaurant,Bar,American Restaurant,Diner,Bakery,Brewery,Café,French Restaurant,Italian Restaurant
22,West Toronto,1,Café,Bar,Coffee Shop,Convenience Store,Thai Restaurant,Italian Restaurant,Sushi Restaurant,Cajun / Creole Restaurant,Restaurant,Pizza Place
24,Central Toronto,1,Café,Coffee Shop,Italian Restaurant,Vegetarian / Vegan Restaurant,Gym,Restaurant,Bakery,Grocery Store,Pub,Museum
25,West Toronto,1,Coffee Shop,Café,Bar,Breakfast Spot,Sushi Restaurant,Grocery Store,Pub,Bakery,Thai Restaurant,Pizza Place


## Cluster 3


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant,Italian Restaurant,Bakery,Theater,Clothing Store,Cocktail Bar
5,Downtown Toronto,2,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Bakery,Restaurant,Gastropub,Cocktail Bar,Grocery Store
8,Downtown Toronto,2,Coffee Shop,Café,Hotel,Gastropub,Theater,Pizza Place,Art Gallery,Concert Hall,Clothing Store,Plaza
10,Downtown Toronto,2,Coffee Shop,Hotel,Café,Park,Japanese Restaurant,Brewery,Gym,Theater,Plaza,Baseball Stadium
13,Downtown Toronto,2,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Theater,Gastropub,Cocktail Bar,Italian Restaurant,Seafood Restaurant
16,Downtown Toronto,2,Coffee Shop,Café,Hotel,Seafood Restaurant,Restaurant,Concert Hall,Gastropub,Japanese Restaurant,Theater,Plaza
34,Downtown Toronto,2,Coffee Shop,Café,Japanese Restaurant,Gastropub,Hotel,Bakery,Park,Restaurant,Seafood Restaurant,Cocktail Bar
36,Downtown Toronto,2,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Theater,Cosmetics Shop,Italian Restaurant,Seafood Restaurant,Sandwich Place


## Cluster 4

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Bookstore,Park,College Quad,College Gym,Gym / Fitness Center,Coffee Shop,Café,Trail,Yoga Studio,Elementary School


## Cluster 5

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,West Toronto,4,Park,Café,Coffee Shop,Italian Restaurant,Sushi Restaurant,Bar,Grocery Store,Brewery,Portuguese Restaurant,Bakery
19,Central Toronto,4,Sushi Restaurant,Pharmacy,Italian Restaurant,Coffee Shop,Bank,Café,Gym Pool,Gym,Lingerie Store,Spa
21,Central Toronto,4,Park,Café,Bank,Coffee Shop,Trail,Burger Joint,Sushi Restaurant,Japanese Restaurant,Bakery,Gym / Fitness Center
23,Central Toronto,4,Coffee Shop,Italian Restaurant,Mexican Restaurant,Café,Restaurant,Thai Restaurant,Diner,Park,Sporting Goods Shop,Food & Drink Shop
26,Central Toronto,4,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pizza Place,Restaurant,Dessert Shop,Indian Restaurant,Gym,Middle Eastern Restaurant
29,Central Toronto,4,Italian Restaurant,Grocery Store,Park,Coffee Shop,Thai Restaurant,Gym,Café,Restaurant,Sushi Restaurant,Bank
31,Central Toronto,4,Coffee Shop,Sushi Restaurant,Thai Restaurant,Italian Restaurant,Park,Grocery Store,Gym,Gym / Fitness Center,Liquor Store,Restaurant
33,Downtown Toronto,4,Coffee Shop,Park,Grocery Store,Juice Bar,Filipino Restaurant,Bank,Bistro,Japanese Restaurant,BBQ Joint,Athletics & Sports
38,East Toronto,4,Park,Pizza Place,Brewery,Coffee Shop,Fast Food Restaurant,Bakery,Sushi Restaurant,Italian Restaurant,French Restaurant,Steakhouse
